<table style="width: 100%; border-style: none">
 <tbody>
  <tr style="border-style: none">
   <td style="border-style: none; width: 1%; text-align: left; font-size: 16px">Institute for Biomedical Imaging<br />Hamburg University of Technology</td>
   <td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
   <td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Dr. Martin Möddel<br />Marija Boberg<br />Mirco Grosser</td>
  </tr>
 </tbody>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computer Graphics</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Exercise 5 - Triangularization</h1>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">summer term 2023</h3>
<hr>

* 📅 Due date: 13.06.2023, 11 a.m.
* You can earn 2 points for each task (8 points in total).

## 0. Get started

The starting point of the exercise is the type `Obj3D` with its plotting routine. An `Obj3D` contains three elements:
* The `name` of the object.
* A list of vertices is stored in an array `vertices` where each entry is an `SVector{3,Float32}` containing the coordinates of a vertex. 
* Each entry in the array `faces` is an `SVector{3,UInt16}` containing the indices of three vertices, which form a triangle. 

In [3]:
using StaticArrays, Plots, LinearAlgebra
gr()


# create a composite type to store the information required to describe a 3D object
abstract type Obj
end

mutable struct Obj3D <: Obj
    name::String
    # a list of 3D vertices 
    vertices::Array{SVector{3,Float32},1}
    # a list of triples
    # each triple references three vertices, which form a triangle
    faces::Array{SVector{3,UInt16},1}

    Obj3D() = new("",SVector{3,Float32}[],SVector{3,UInt16}[])
end

import Plots.plot

function plot(obj::Obj;xlim=(-2,2),ylim=(-2,2),verbose::Bool=true)
    p = plot(leg=false, xlim=xlim,ylim=ylim)
    for face in obj.faces
        x = [obj.vertices[i][1] for i in face[[1,2,3,1]]]
        y = [obj.vertices[i][2] for i in face[[1,2,3,1]]]
    
        plot!(x,y,line=(:black,),aspect_ratio=1)
    end
    
    verbose && println("$(length(obj.faces)) faces are drawn")
    return p
end

plot (generic function with 5 methods)

# Marching Triangularization

We want to implement the method for triangularization you discussed in the lecture for a sphere. The sphere is given as an implicit surface by
\begin{equation}
f(x,y,z) = x^2+y^2+z^2-r^2.
\end{equation}
Choose $r=1$.

## 1. Gradient Projection

* Implement a function `gradientProjection(p::SVector{3,Float32},f::Function,gradf::Function;ε=1e-5)` that takes a point `p`  and projects it onto the surface described by a function `f(x,y,z)`. In addition, you need a function `gradf(x,y,z)`, which returns the gradient of `f` at a given point. Here `ε` is the tolerance used for stopping the algorithm.

* Implement a function `surfaceNormal(p::SVector{3,Float32},gradf::Function)`  to calculate the surface normal at a given point. Also implement a function `tangentVectors(n::SVector{3,Float32})` to get tangent vectors orthogonal to a normal vector `n`.

Test your functions by applying them to the point $\mathbf{p} = (1,1,1)$ and the sphere given by `f` and its derivative `gradf`:
```julia
using Test
@testset "Gradient Projection" begin
  p = @SVector ones(Float32,3)
  u = gradientProjection(p,f,gradf)
  n = surfaceNormal(p,gradf)
  t = tangentVectors(n)

  @test isapprox(u,Float32[0.57735026, 0.57735026, 0.57735026])
  @test isapprox(n,[0.57735026, 0.57735026, 0.57735026])
  @test isapprox(t[1],Float32[0.70710677, -0.70710677, 0.0])
  @test isapprox(t[2],Float32[0.40824828, 0.40824828, -0.81649655])
end;
```

In [4]:
function gradientProjection(p::SVector{3,Float32}, f::Function, gradf::Function; ε=1e-5)
    u = []
    push!(u, p) #u_0

    #first u_k+1 calculation
    push!(u, u[1] - f(u[1]...) / dot(gradf(u[1]...), gradf(u[1]...)) .* gradf(u[1]...))
    if norm(u[2]-u[1]) < ε
        return u[2]
    end    

    #repetition until norm is small enough
    while true
        u[1] = u[2] - f(u[2]...) / dot(gradf(u[2]...), gradf(u[2]...)) .* gradf(u[2]...)
        if norm(u[1]-u[2]) < ε
            return u[1]
        end
        
        #ping-pong between u_1 and u_2
        u[2] = u[1] - f(u[1]...) / dot(gradf(u[1]...), gradf(u[1]...)) .* gradf(u[1]...)
        if norm(u[2]-u[1]) < ε
            return u[2]
        end
    end
end

function surfaceNormal(p::SVector{3,Float32},gradf::Function) 
    return normalize(gradf(p...))
end

function tangentVectors(n::SVector{3,Float32})
    t = Array{SVector{3, Float32}}(undef,2)
    
    if n[1] > 0.5 || n[2] > 0.5
        t[1] = normalize(SVector{3, Float32}(n[2], -n[1], 0))
    else
        t[1] = normalize(SVector{3, Float32}(-n[3], 0, n[1]))
    end
    
    t[2] = cross(n, t[1])
    return t
end

tangentVectors (generic function with 1 method)

In [5]:
using Test

@testset "Gradient Projection" begin
    # function f(x, y, z) for the sphere
    f(x, y, z) = x^2 + y^2 + z^2 - 1.0f0

    # gradient of f(x, y, z) for the sphere
    gradf(x, y, z) = @SVector([2 * x, 2 * y, 2 * z])

    # test gradient projection, surface normal, and tangent vectors functions
    @testset "Sphere" begin
        p = @SVector([1.0f0, 1.0f0, 1.0f0])
        u = gradientProjection(p, f, gradf)
        n = surfaceNormal(p, gradf)
        t = tangentVectors(n)

        @test isapprox(u, @SVector([0.57735026, 0.57735026, 0.57735026]))
        @test isapprox(n, @SVector([0.57735026, 0.57735026, 0.57735026]))
        @test isapprox(t[1], @SVector([0.70710677, -0.70710677, 0.0]))
        @test isapprox(t[2], @SVector([0.40824828, 0.40824828, -0.81649655]))
    end
end

Test Summary:       | Pass  Total  Time
Gradient Projection |    4      4  1.2s


Test.DefaultTestSet("Gradient Projection", Any[Test.DefaultTestSet("Sphere", Any[], 4, false, false, true, 1.68624378176e9, 1.686243782955e9)], 0, false, false, true, 1.686243781728e9, 1.686243782955e9)

## 2. Hexagon Initialization 

Implement a function 
```julia
hexagonInitialization(sp::SVector{3,Float32},δ::Float32,f::Function,gradf::Function)
```  
which returns the primary outer front polygon.

Choose a starting point `sp` and project it onto the surface. Then build a hexagon, with side length `δ`, in the tangent space of the first vertex and project it onto the surface. The resulting hexagon describes the primary outer front polygon. Save the vertices and triangles to an object `Obj3D` and return the latter as well.

* In order to store all relevant information, use the following `type FrontPolygon`.
* Adding and deleting a vertex can be achieved with the convenience functions implemented below.

Plot the result for the sphere given by `f` and its derivative `gradf`.
```julia
δ = 0.2f0
sp = SVector{3,Float32}(0,2,4)
sphere, fp = hexagonInitialization(sp,δ,f,gradf)
plot(sphere)
```

In [6]:
mutable struct FrontPolygon
    #sorted vertices
    vertices::Array{SVector{3,Float32},1}
    # corresponding surface normals
    normals::Array{SVector{3,Float32},1}
    # corresponding tangent vectors
    tangent1::Array{SVector{3,Float32},1}
    tangent2::Array{SVector{3,Float32},1}
    # array for distance check query
    distCheck::Array{Bool,1}
    
    FrontPolygon() = new(SVector{3,Float32}[],SVector{3,Float32}[],SVector{3,Float32}[],SVector{3,Float32}[],Bool[])
end

# add a new vertex to a front polygon and calculate normal and tangent vectors
# gradf is the gradient of the function describing an implicit surface
function addVertex(fp::FrontPolygon,v::SVector{3,Float32},gradf::Function)
    push!(fp.vertices,v)
    n = surfaceNormal(v,gradf)
    push!(fp.normals,n)
    t = tangentVectors(n)
    push!(fp.tangent1,t[1])
    push!(fp.tangent2,t[2])
    push!(fp.distCheck,true)
end

# insert a new vertex to a front polygon at index ind. Calculate normal and tangent vectors.
# gradf is the gradient of the function describing an implicit surface
function addVertex(fp::FrontPolygon,v::SVector{3,Float32},ind::Int,gradf::Function)
    insert!(fp.vertices,ind,v)
    n = surfaceNormal(v,gradf)
    insert!(fp.normals,ind,n)
    t = tangentVectors(n)
    insert!(fp.tangent1,ind,t[1])
    insert!(fp.tangent2,ind,t[2])
    insert!(fp.distCheck,ind,true)
end

# copy a vertex with normal vector, tangential vectors and distance check from one front
# polygon (fp2 at index ind2) to another (fp at index ind)
function copyVertex(fp::FrontPolygon,fp2::FrontPolygon,ind::Int,ind2::Int)
    insert!(fp.vertices,ind,fp2.vertices[ind2])
    insert!(fp.normals,ind,fp2.normals[ind2])
    insert!(fp.tangent1,ind,fp2.tangent1[ind2])
    insert!(fp.tangent2,ind,fp2.tangent2[ind2])
    insert!(fp.distCheck,ind,fp2.distCheck[ind2])
end

# delete a vertex with its corresponding values at index ind
function deleteVertex(fp::FrontPolygon,ind::Int)
    deleteat!(fp.vertices,ind)
    deleteat!(fp.normals,ind)
    deleteat!(fp.tangent1,ind)
    deleteat!(fp.tangent2,ind)
    deleteat!(fp.distCheck,ind)
end

deleteVertex (generic function with 1 method)

In [7]:
function hexagonInitialization(sp::SVector{3,Float32},δ::Float32,f::Function,gradf::Function)
    #initial point on surface
    point_0   = gradientProjection(sp,f,gradf)
    norm_0    = surfaceNormal(point_0,gradf)
    tangent_0 = tangentVectors(norm_0)

    #first front polygon
    front = FrontPolygon()
    
    for i=1:6
        # vertices q1, . . . , q6 of the initial hexagon
        qi = SVector{3,Float32}(point_0 + δ*cos((i-1)*pi/3).*tangent_0[1] + δ*sin((i-1)*pi/3).*tangent_0[2])

        # project to get point p1 to p6 on surface
        hex_point = gradientProjection(qi,f,gradf)

        # add vertex p1 to p6 to front polygon
        addVertex(front, hex_point, gradf)
    end
    
    obj = Obj3D()
    #center of hexagon p0
    push!(obj.vertices, point_0)
    
    for i=1:6
        #outer vertices of hexagon Π_0 = {p1, . . . , p6} on the surface
        push!(obj.vertices, front.vertices[i])

        #faces of hexagon (p0, p1, p2), (p0, p2, p3), (p0, p3, p4), (p0, p4, p5), (p0, p5, p6), and (p0, p6, p1)
        push!(obj.faces, [1, i+1, mod(i,6)+2])
    end
    
    return obj, front
end

hexagonInitialization (generic function with 1 method)

In [9]:
# to test above code
δ = 0.2f0
sp = SVector{3,Float32}(0, 2, 4)

f(x, y, z) = x^2 + y^2 + z^2 - 1.0f0
gradf(x, y, z) = @SVector([2 * x, 2 * y, 2 * z])

sphere, fp = hexagonInitialization(sp, δ, f, gradf)
plot(sphere)


6 faces are drawn


## 3. Angles of the Front Polygon

Implement a function `frontAngles(fp::FrontPolygon)`, which calculates the front angles of a front polygon.

To calculate the coordinate transformation to the local coordinate system of each vertex ($\mathbf v = \mathbf p_i + \eta \mathbf n_i + \tau \mathbf t_{i1} +\vartheta \mathbf t_{i2}$), you may use the function `localCoords`. Analogously, the inverse transform can be computed using the function `globalCoords`:
```julia
hSpace(v::SVector{3,Float32}) = SVector{4,Float32}(v[1],v[2],v[3],1)
nSpace(v::SVector{4,Float32}) = SVector{3,Float32}(v[1]/v[4],v[2]/v[4],v[3]/v[4])

function localCoords(v::SVector{3,Float32},p::SVector{3,Float32},n::SVector{3,Float32},t1::SVector{3,Float32},t2::SVector{3,Float32})
    A = SMatrix{4,4,Float32}(1,0,0,0, 0,1,0,0, 0,0,1,0, -p[1],-p[2],-p[3],1)
    B = SMatrix{4,4,Float32}(n[1],t1[1],t2[1],0, n[2],t1[2],t2[2],0, n[3],t1[3],t2[3],0, 0,0,0,1)
    return nSpace(B*A*hSpace(v))
end

function globalCoords(v::SVector{3,Float32},p::SVector{3,Float32},n::SVector{3,Float32},t1::SVector{3,Float32},t2::SVector{3,Float32})
    A = SMatrix{4,4,Float32}(1,0,0,0, 0,1,0,0, 0,0,1,0, p[1],p[2],p[3],1)
    B = SMatrix{4,4,Float32}(n[1],t1[1],t2[1],0, n[2],t1[2],t2[2],0, n[3],t1[3],t2[3],0, 0,0,0,1)'
    return nSpace(A*B*hSpace(v))
end
```

Compute the front angles of the first front polygon of the sphere by testing your code with
```julia
@testset "Front Angles" begin
    
  @test isapprox(frontAngles(fp),4.171891 .* ones(6))
end;
```

In [23]:
using LinearAlgebra

hSpace(v::SVector{3,Float32}) = SVector{4,Float32}(v[1], v[2], v[3], 1)
nSpace(v::SVector{4,Float32}) = SVector{3,Float32}(v[1] / v[4], v[2] / v[4], v[3] / v[4])


function localCoords(v::SVector{3,Float32}, p::SVector{3,Float32}, n::SVector{3,Float32}, t1::SVector{3,Float32}, t2::SVector{3,Float32})
    A = SMatrix{4,4,Float32}(1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, -p[1], -p[2], -p[3], 1)
    B = SMatrix{4,4,Float32}(n[1], t1[1], t2[1], 0, n[2], t1[2], t2[2], 0, n[3], t1[3], t2[3], 0, 0, 0, 0, 1)
    return nSpace(B * A * hSpace(v))
end

function globalCoords(v::SVector{3,Float32}, p::SVector{3,Float32}, n::SVector{3,Float32}, t1::SVector{3,Float32}, t2::SVector{3,Float32})
    A = SMatrix{4,4,Float32}(1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, p[1], p[2], p[3], 1)
    B = SMatrix{4,4,Float32}(n[1], t1[1], t2[1], 0, n[2], t1[2], t2[2], 0, n[3], t1[3], t2[3], 0, 0, 0, 0, 1)'
    return nSpace(A * B * hSpace(v))
end

function frontAngles(fp::FrontPolygon)
    angles = Float32[]
    num_vertices = length(fp.vertices)

    for i = 1:num_vertices
        v1 = fp.vertices[mod1(i-1, num_vertices)]
        v2 = fp.vertices[mod1(i+1, num_vertices)]

        local_coords_v1 = localCoords(v1, fp.vertices[i], fp.normals[i], fp.tangent1[i], fp.tangent2[i])
        local_coords_v2 = localCoords(v2, fp.vertices[i], fp.normals[i], fp.tangent1[i], fp.tangent2[i])

        phi1 = atan(local_coords_v1[3], local_coords_v1[2])
        phi2 = atan(local_coords_v2[3], local_coords_v2[2])

        angle = (phi2 >= phi1) ? (phi2 - phi1) : (2*pi + phi2 - phi1)
        push!(angles, angle)
    end

    return angles
end



frontAngles (generic function with 1 method)

In [24]:

using Test

@testset "Front Angles" begin
    @test isapprox(frontAngles(fp), 4.171891 .* ones(6))
end


Test Summary: | Pass  Total  Time
Front Angles  |    1      1  0.1s


Test.DefaultTestSet("Front Angles", Any[], 1, false, false, true, 1.686244203961e9, 1.686244204071e9)

## 4. Triangle Creation

Create triangles around the point with minimal front angle.

Implement a function `triangleCreation` that keeps adding triangles to the front polygon. The method should stop as soon as there exist two vertices with a distance smaller then $\delta /2$, which are neither neighbours nor neighbours of neighbours. This condition can be tested using the method `breakCondition` implemented below.

For storing new triangles in an existing `Obj3D`, `findVertex` will give you the index of the object vertices.

The matrix for a rotation around the normal vector in the local coordinate system of the vertices by $\alpha$ can be calculated with the method `rotation_n`.

* Test your code with 
```julia
sphere = Obj3D()
fp = hexagonInitialization(sphere,sp,δ,f,gradf)
triangleCreation(fp,sphere,δ,f,gradf)
plot(sphere)
```

* Also test your code with a torus for which the implicit surface is given by
\begin{equation}
g(x,y,z) = \left(\sqrt{x^2+y^2}-R\right)^2+z^2-r^2
\end{equation}
with $R=3$ and $r=1$
```julia
torus = Obj3D()
fpT = hexagonInitialization(torus,sp,δ,g,gradg)
triangleCreation(fpT,torus,δ,g,gradg)
plot(torus,xlim=(-4,4),ylim=(-4,4))
```

In [12]:
function findVertex(obj::Obj3D,v::SVector{3,Float32})
    for i=1:length(obj.vertices)
        if isapprox(obj.vertices[i],v,atol=1e-5)
            return i
        end
    end
    return 0
end

function rotation_n(α)
    return SMatrix{4,4,Float32}(1,0,0,0 ,0,cos(α),sin(α),0 ,0,-sin(α),cos(α),0 ,0,0,0,1)
end

function breakCondition(fp::FrontPolygon,δ::Float32)
    for i=1:length(fp.vertices)
        if i==1
            J = length(fp.vertices)-2
        elseif i==2
            J = length(fp.vertices)-1
        else
            J = length(fp.vertices)
        end
        for j=i+2:J
            if norm(fp.vertices[i]-fp.vertices[j],2) < δ/2
                return true
            end
        end
    end
    
    return false
end

breakCondition (generic function with 1 method)